# Octree Mesh Creation

 <img align="right" width="400px" src="./images/octree_mesh_creation.png">
 
This application lets users create `Octree` mesh objects with local refinements using the [SimPEG.discretize](http://discretize.simpeg.xyz/en/main/) package. The refinements can be performed on any number of objects.

See the [Methodology](#Methodology) section to learn more about how the algorithm works.

New user? Visit the [**Getting Started**](https://geoapps.readthedocs.io/en/latest/content/installation.html) page.

## Application
The following sections provide details on the different parameters controlling the application. Interactive widgets shown below are for demonstration purposes only.

In [1]:
from geoapps.create.octree_mesh import OctreeMesh

app = OctreeMesh(h5file=r"../../../assets/FlinFlon.geoh5")
app()

## Project Selection

Select and connect to an existing **geoh5** project file containing objects with vertices. 

OR 

Select a `*.ui.json` input file to re-load parameters from. See the [Input ui.json](#Input-ui.json) section for details.

In [2]:
app.project_panel

See the [Project Panel](base_application.ipynb#Project-Panel) page for more details.

## Core hull extent

List of objects available to define the core region of the mesh. Padding parameters are added beyond the convex hull of the selected object. 

In [3]:
app.objects

Dropdown(description='Core hull extent:', index=69, options=(['', None], ['CoordinateTransformation/Data_TEM_p…

## Mesh Parameters

 <img align="right" width="40%" src="./images/Octree_core_depth.png">
 
### Minimum depth

Minimum thickness of the mesh below the [core hull extent](#Core-hull-extent). See the [Methodology](#Methodology) section for details.

In [4]:
app.depth_core

FloatText(value=500.0, description='Minimum depth (m)', style=DescriptionStyle(description_width='initial'))

### Core cell sizes

#### Easting

Minimum cell size (m) along the x-axis


In [5]:
app.u_cell_size

FloatText(value=25.0, description='Easting', style=DescriptionStyle(description_width='initial'))

#### Northing

Minimum cell dimension (m) along the y-axis

In [6]:
app.v_cell_size

FloatText(value=25.0, description='Northing', style=DescriptionStyle(description_width='initial'))

#### Vertical

Minimum cell dimension (m) along the z-axis

In [7]:
app.w_cell_size

FloatText(value=25.0, description='Vertical', style=DescriptionStyle(description_width='initial'))

### Padding distances

Minimum padding distance (m) requested around the core region. 


#### Horizontal

Padding distance added around the core region on the XY-plane.

In [8]:
app.horizontal_padding

FloatText(value=1000.0, description='Horizontal (m)', style=DescriptionStyle(description_width='initial'))

#### Vertical

Padding distance added above and below the core region.

In [9]:
app.vertical_padding


FloatText(value=1000.0, description='Vertical (m)', style=DescriptionStyle(description_width='initial'))

## Refinements

Blocks of refinement parameters controling where small cells are required. Overlapping regions of refinement will default to the smaller cell size. Cells outside of the regions of refinement are allowed to increase in size (double) as quickly as possible to fill the remaining volume.

**Note**: The application defaults to two blocks of refinement, labeled as A and B. [See notes below](#Input-ui.json) to learn how to add more blocks of refinement.

In [10]:
app.refinement_list.children[0]

### Object

Reference object with 3D coordinates (`vertices` or `centroids`) used to refine the octree mesh. 

In [11]:
app.refinement_list.children[0].children[1]

Dropdown(description='Object', index=69, options=(['', None], ['CoordinateTransformation/Data_TEM_pseudo3DEPSG…

### Levels

List of integers defining the minimum number of cells at each octree level [$N_{level 1}$, $N_{level 2}$, $N_{level 3}$, ...]. [See examples below](#Radial)

In [12]:
app.refinement_list.children[0].children[2]

Text(value='4.0, 4.0, 4.0', description='Levels', style=DescriptionStyle(description_width='initial'))

### Type

The type of refinement to be performed on the selected object vertices and number of cells per octree level.

In [13]:
app.refinement_list.children[0].children[3]

Dropdown(description='Type', index=1, options=('surface', 'radial'), style=DescriptionStyle(description_width=…

#### Radial

Octree cells specified at each [Level](#Levels) are added as concentric shells around the selected `Object` `vertices`.

 <img align="center" width="75%" src="./images/Octree_radial.png">
 
 
#### Surface

Octree cells specified at each [Level](#Levels) are added at depth as layers of cells. The layers of cells are defined by a 2D Delaunay triangulation based on the `Object` `vertices`. The [maximum distance](#Max-distance) parameter can be use to limit the horizontal extent of the refinement. 

 <img align="center" width="75%" src="./images/Octree_surface.png">

### Max distance

Maximum refinement distance away from the input vertices. Cell sizes are allowed to increase quickly beyond this distance. 

**Note**: This parameter is useful to avoid a fine refinement occuring between remote vertices as shown above.

In [14]:
app.refinement_list.children[0].children[4]

FloatText(value=1000.0, description='Distance', style=DescriptionStyle(description_width='initial'))

## Output panel

An octree mesh is created and exported directly to the target workspace by clicking on the export button.

In [15]:
app.output_panel

### (Optional) Geoscience ANALYST Pro - Live link
See [Output Panel](base_application.ipynb#Output-Panel) base applications.

In [16]:
import os
app._working_directory = os.getcwd()
out = app.save_json_params(app.ga_group_name.value, app.params.default_ui_json.copy())

## Methodology

This section provides algorithmic details about the mesh creation. The entire process can be broken down into three main parts:

### Step 1- Mesh creation


The first stage relies on the [discretize.utils.mesh_builder_xyz](http://discretize.simpeg.xyz/en/main/api/generated/discretize.utils.mesh_builder_xyz.html?highlight=xyz#discretize-utils-mesh-builder-xyz)
method to define the outer limits of the mesh. The horizontal and vertical extent are computed based on the selected entity [Core Hull Extent](#Core-hull-extent), [padding distances](#Padding-distances) and [Minimum Depth](#Minimum-depth) (vertical only). Limits are rounded up such that the number of core cells along each dimension is in power of 2 (i.e. 64, 128, ...), as required for octree discretization.

The example below demonstrates this process with the default parameters:

 <img align="right" width="75%" src="./images/Octree_padding_distance.png">
 
 
**Horizontal extent (East-West):**
   
    Input: 5600 m (hull) + 2*1000 m (pad) = 7600 m 

    Round: 7600 m / 25 m/cell = 304 cells -> 512 cells

    Final: 512 cells * 25 m/cell = 12800 m


**Vertical extent:**

    Input: 46 m (hull) + 2*1000 m (pad) + 500 (core) m = 2546 m 

    Round: 2546 m / 25 m/cell = 102 cells -> 128 cells

    Final: 128 cells * 25 m/cell = 3,200 m

 
### Step 2- Refinements 

Once the extent of the mesh has been defined, the program can proceed with the [Refinements](#Refinements) prescribed by the user. The routine makes use of the [discretize.utils.refine_tree_xyz](http://discretize.simpeg.xyz/en/main/api/generated/discretize.utils.refine_tree_xyz.html#discretize-utils-refine-tree-xyz) utility function. The order of refinements is not important as only the smallest cell octree found at a given position is used. 

### Step 3 - Finalization

Once all the refinements have been added, the [TreeMesh.finalize()](http://discretize.simpeg.xyz/en/main/api/generated/discretize.TreeMesh.html?highlight=finalize#discretize.TreeMesh.finalize) routine is called for the final construction of the mesh. Cells in regions outside of the refinements are allowed to expand as quickly as possible. Et voila!


## Input ui.json 

This application relies on a [structured json](./Octree_Mesh.ui.json) file to store and run the program.

The same input `ui.json` file can be used to run the program from command line:

```
activate geoapps
python -m geoapps.create.octree_mesh input.ui.json
```

or directly from [Geoscience ANALYST Pro](#Geoscience-ANALYST-Pro) v3.3.1 or higher.
The `ui.json` can also be used to re-load the parameters from a previous run by selecting the `ui.json` file from the [Project Selection](#Project-Selection) widget, instead of a `geoh5` file. 

### Notes 

 <img align="right" width="50%" src="./images/refinement_block.png">
 
- Additional refinements can be performed by adding new blocks of parameters to the default [input ui.json file](./Octree_Mesh.ui.json):
    - For each additional refinement blocks, a unique name must be provided (C, D, etc) 
    - Each block must include all [refinement parameters](#Refinements).
    - Changes made will be displayed after re-loading the ui.json from the [Project Selection](#Project-Selection) panel. 


 



### Geoscience ANALYST Pro v3.3.1

Users with an active Geoscience ANALYST Pro license (v3.3.1) can execute this application directly from an active session with a drag & drop of a `*.ui.json`file to the `Viewport`. 

![GAPro_Octree](images/GA_pro_octree.gif "ga_pro")

Need help? Contact us at support@mirageoscience.com